In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd 
from keras.layers import LSTM,Dropout,Dense
from keras.layers import SimpleRNN
from keras.layers import GRU
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

2023-04-03 10:10:10.448452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Read data**


In [2]:
df=pd.read_csv("/Users/rahuljauhari/Desktop/research runoff/final destination/merged_imd.csv")
df.drop(columns=['Unnamed: 0'],inplace=True)


In [3]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)
monthly_mean = df.resample('M').mean()

**Actual value**


In [4]:
df_actual=pd.read_excel("/Users/rahuljauhari/Desktop/research runoff/Calibrated and Validated.xlsx")
# select last column
observed_runnoff=df_actual['observed']
# observed_runnoff.head()


**Normalization**


In [5]:
from scipy.stats import zscore
def func(name):
    x=0
    y=0
    inv= 0
    if name=='zscore':
        x_norm = zscore(monthly_mean)
        y_norm = zscore(observed_runnoff)
        x_norm[x_norm > 3] = 2.8
        x_norm[x_norm < -3] = -2.8
        y_norm[y_norm >3] = 2.8
        y_norm[y_norm < -3] = -2.8
        x=x_norm
        y=y_norm
    if name=='StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x_scaled[x_scaled > 3] = 2.8
        x_scaled[x_scaled < -3] = -2.8
        y_scaled[y_scaled >3] = 2.8
        y_scaled[y_scaled < -3] = -2.8
        x=      x_scaled  
        y=y_scaled
        inv = scaler
        
    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0,1))
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x=      x_scaled  
        y=y_scaled
        inv = scaler
    return x,y,inv


**Metric**

In [6]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

In [7]:
activation = ['relu','linear','sigmoid']
optimizer =['adam','rmsprop','sgd']
preprocess = ['MinMaxScaler','StandardScaler','zscore']

In [8]:
from sklearn.model_selection import train_test_split
# adam optimizer import
from keras.optimizers import Adam, RMSprop, SGD
x,y,inv = func('MinMaxScaler')
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=False)
# validation_data 10% of train data
x_train, x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.1,shuffle=False)

In [10]:
# keras tuner for gru rnn
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras import regularizers
from keras_tuner.engine.hyperparameters import HyperParameters
from keras.optimizers import Adam, RMSprop, SGD

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.GRU(units=hp.Int('units',min_value=32,max_value=512,step=32),activation=hp.Choice('act_' , values=['relu', 'sigmoid','linear','tanh']),return_sequences=True,input_shape=(x_train.shape[1],1)))
    model.add(layers.Dense(hp.Int('units',min_value=32,max_value=512,step=32),activation=hp.Choice('act_' , values=['relu', 'sigmoid','linear','tanh'])))
    model.add(Dropout(hp.Float('dropout_1', 0, 0.5, step=0.1, default=0.2)))
    model.add(layers.SimpleRNN(units=hp.Int('units',min_value=32,max_value=512,step=32),activation=hp.Choice('act_' , values=['relu', 'sigmoid','linear','tanh']),return_sequences=False))
    model.add(layers.Dense(1,activation=hp.Choice('act_' , values=['relu', 'sigmoid','linear','tanh']),kernel_regularizer=regularizers.l2(hp.Float('l2', 0, 0.5, step=0.1, default=0.2))))
    model.add(Dropout(hp.Float('dropout_1', 0, 0.5, step=0.1, default=0.2)))
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])

    if hp_optimizer == 'sgd':
        optimizer = SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=hp_lr)
    else:
        optimizer = Adam(learning_rate=hp_lr)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    return model
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner = RandomSearch( build_model, objective='mse', max_trials=10, executions_per_trial=1,directory='project', project_name='GRU_RNN')
tuner.search(x_train, y_train, epochs=100,verbose=0,validation_data=(x_val, y_val),callbacks = [early_stopping])


INFO:tensorflow:Reloading Tuner from project/GRU_RNN/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [11]:
# print best model summary
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 174, 128)          50304     
                                                                 
 dense (Dense)               (None, 174, 128)          16512     
                                                                 
 dropout (Dropout)           (None, 174, 128)          0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
 dropout_1 (Dropout)         (None, 1)                 0         
                                                                 
Total params: 99,841
Trainable params: 99,841
Non-traina

In [13]:
best_model = tuner.get_best_models()[0]
from sklearn.metrics import r2_score
print(type(best_model.predict(x_train)))
print("train ",r2_score(y_train, best_model.predict(x_train)))
print("val ",r2_score(y_val, best_model.predict(x_val)))
print("test ",r2_score(y_test, best_model.predict(x_test)))
best_model.summary()
# optimizer used in best model
print(best_model.optimizer.get_config())
# activation used in best model
best_model.layers[0].get_config()['activation']

10/10 [==============================] - 1s 41ms/step
<class 'numpy.ndarray'>
10/10 [==============================] - 0s 40ms/step
train  0.4481372708702872
2/2 [==============================] - 0s 13ms/step
val  0.5744090955468866
5/5 [==============================] - 0s 38ms/step
test  0.45550361964457664
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 174, 128)          50304     
                                                                 
 dense (Dense)               (None, 174, 128)          16512     
                                                                 
 dropout (Dropout)           (None, 174, 128)          0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense_1

'tanh'

In [14]:
# dropout 
best_model.layers[2].get_config()['rate']

0.0

In [16]:
# regular
best_model.layers[4].get_config()['kernel_regularizer']

{'class_name': 'L2', 'config': {'l2': 0.4000000059604645}}

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score
# for act in activation:
#     for opt in optimizer:
#         for pre in preprocess:
#             x,y,inv_scaler= func(pre)
#             X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=False)
#             model = keras.Sequential()
#             model.add(GRU(64,return_sequences=True, activation=act, input_shape=(X_train.shape[1],1)))
#             model.add(Dense(64))
#             model.add(Dropout(0.5))
#             model.add(SimpleRNN(64, activation=act))
#             model.add(Dense(64))
#             model.add(Dropout(0.5))
#             model.add(layers.Dense(1))
#             model.compile(optimizer=opt, loss='mse')
#             model.fit(X_train, y_train, batch_size=10, epochs=100,shuffle=False, use_multiprocessing=True,verbose=0,validation_split=0.1)
#             y_pred = model.predict(X_test)
#             y_pred_train = model.predict(X_train)
#             try:
#                 _ = pd.DataFrame({'pre':pre,'act':act,'opt':opt,'r2_train':r2_score(y_train,y_pred_train),'r2_test':r2_score(y_test,y_pred)},index=[0])
#                 # _ = pd.DataFrame({'pre':pre,'act':act,'opt':opt,'rmse_train':rmse1(y_train,y_pred_train),'rmse_test':rmse1(y_test,y_pred),'kge_train':kge1(y_train,y_pred_train),'kge_test':kge1(y_test,y_pred),'r2_train':r21(y_train,y_pred_train),'r2_test':r21(y_test,y_pred)},index=[0])
#                 _.to_csv('/Users/rahuljauhari/Desktop/research runoff/results1/imd_rnn_gru1.csv',mode='a',header=True)
#             except Exception as e:
#                 print(e)